In [1]:
# -----------------------------------------------------------------------------
# this runs the Cape Cod site flow case for the Harvey and Garabedian bacterial
# injection experiment from Maxwell, et al, 2007.
# -----------------------------------------------------------------------------

import os, sys
from set_demo_defaults import *

import matplotlib.pyplot as plt
from parflow import Run
from parflow.tools.fs import cp, chdir, mkdir, get_absolute_path, rm
from parflow.tools.io import read_pfb, write_pfb, ParflowBinaryReader
from parflow.tools.compare import pf_test_file
from parflow.tools.hydrology import compute_hydraulic_head
from plots import plot_domain_corr_rnd
import numpy as np

run_name = "harvey_flow.1"
dir_name = get_absolute_path("test_output/harvey_flow")
mkdir(dir_name)
def correlated_random_field(alpha_vG):
    #hflow = Run(run_name, __file__)
    hflow = Run(run_name)
    
    # -----------------------------------------------------------------------------
    # File input version number
    # -----------------------------------------------------------------------------
    
    hflow.FileVersion = 4
    
    # -----------------------------------------------------------------------------
    # Process Topology
    # -----------------------------------------------------------------------------
    
    hflow.Process.Topology.P = 1
    hflow.Process.Topology.Q = 1
    hflow.Process.Topology.R = 1
    
    # -----------------------------------------------------------------------------
    # Computational Grid
    # -----------------------------------------------------------------------------
    hflow.ComputationalGrid.Lower.X = 0.0
    hflow.ComputationalGrid.Lower.Y = 0.0
    hflow.ComputationalGrid.Lower.Z = 0.0
    
    hflow.ComputationalGrid.DX = 0.34
    hflow.ComputationalGrid.DY = 0.34
    hflow.ComputationalGrid.DZ = 0.038
    
    hflow.ComputationalGrid.NX = 50
    hflow.ComputationalGrid.NY = 1
    hflow.ComputationalGrid.NZ = 100
    
    # -----------------------------------------------------------------------------
    # The Names of the GeomInputs
    # -----------------------------------------------------------------------------
    
    hflow.GeomInput.Names = "domain_input upper_aquifer_input"
    
    # -----------------------------------------------------------------------------
    # Domain Geometry Input
    # -----------------------------------------------------------------------------
    
    hflow.GeomInput.domain_input.InputType = "Box"
    hflow.GeomInput.domain_input.GeomName = "domain"
    
    # -----------------------------------------------------------------------------
    # Domain Geometry
    # -----------------------------------------------------------------------------
    hflow.Geom.domain.Lower.X = 0.0
    hflow.Geom.domain.Lower.Y = 0.0
    hflow.Geom.domain.Lower.Z = 0.0
    
    hflow.Geom.domain.Upper.X = 17.0
    hflow.Geom.domain.Upper.Y = 10.2
    hflow.Geom.domain.Upper.Z = 3.8
    
    hflow.Geom.domain.Patches = "left right front back bottom top"
    
    # -----------------------------------------------------------------------------
    # Upper Aquifer Geometry Input
    # -----------------------------------------------------------------------------
    
    hflow.GeomInput.upper_aquifer_input.InputType = "Box"
    hflow.GeomInput.upper_aquifer_input.GeomName = "upper_aquifer"
    
    # -----------------------------------------------------------------------------
    # Upper Aquifer Geometry
    # -----------------------------------------------------------------------------
    
    hflow.Geom.upper_aquifer.Lower.X = 0.0
    hflow.Geom.upper_aquifer.Lower.Y = 0.0
    hflow.Geom.upper_aquifer.Lower.Z = 0.0
    
    hflow.Geom.upper_aquifer.Upper.X = 17.0
    hflow.Geom.upper_aquifer.Upper.Y = 10.2
    hflow.Geom.upper_aquifer.Upper.Z = 3.8
    
    # -----------------------------------------------------------------------------
    # Perm
    # -----------------------------------------------------------------------------
    
    hflow.Geom.Perm.Names = "upper_aquifer"
    
    # we open a file, in this case from PEST to set upper and lower kg and sigma
    #cp("$./stats4.txt")
    chdir(".")
    
    file = open("stats4.txt", "r")
    lines = file.readlines()
    kgu = float(lines[0])
    varu = float(lines[1])
    
    ## we use the parallel turning bands formulation in ParFlow to simulate
    ## GRF for upper and lower aquifer
    
    hflow.Geom.upper_aquifer.Perm.Type = "TurnBands"
    hflow.Geom.upper_aquifer.Perm.LambdaX = 3.60
    hflow.Geom.upper_aquifer.Perm.LambdaY = 3.60
    hflow.Geom.upper_aquifer.Perm.LambdaZ = 0.19
    hflow.Geom.upper_aquifer.Perm.GeomMean = 112.00
    hflow.Geom.upper_aquifer.Perm.Sigma = 0.48989794
    hflow.Geom.upper_aquifer.Perm.NumLines = 150
    hflow.Geom.upper_aquifer.Perm.RZeta = 5.0
    hflow.Geom.upper_aquifer.Perm.KMax = 100.0000001
    hflow.Geom.upper_aquifer.Perm.DelK = 0.2
    hflow.Geom.upper_aquifer.Perm.Seed = 33333
    hflow.Geom.upper_aquifer.Perm.LogNormal = "Log"
    hflow.Geom.upper_aquifer.Perm.StratType = "Bottom"
    
    # -----------------------------------------------------------------------------
    # uncomment the lines below to run parallel gaussian instead
    # of parallel turning bands
    
    # hflow.Geom.upper_aquifer.Perm.Type = "ParGauss"
    
    # hflow.Geom.upper_aquifer.Perm.Seed = 1
    # hflow.Geom.upper_aquifer.Perm.MaxNPts = 70.0
    # hflow.Geom.upper_aquifer.Perm.MaxCpts = 20
    
    
    # pfset lower aqu and upper aq stats to pest/read in values
    # -----------------------------------------------------------------------------
    
    hflow.Perm.TensorType = "TensorByGeom"
    
    hflow.Geom.Perm.TensorByGeom.Names = "domain"
    
    hflow.Geom.domain.Perm.TensorValX = 1.0
    hflow.Geom.domain.Perm.TensorValY = 1.0
    hflow.Geom.domain.Perm.TensorValZ = 1.0
    
    # -----------------------------------------------------------------------------
    # Specific Storage
    # -----------------------------------------------------------------------------
    # specific storage does not figure into the impes (fully sat) case but we still
    # need a key for it
    
    hflow.SpecificStorage.Type = "Constant"
    hflow.SpecificStorage.GeomNames = ""
    hflow.Geom.domain.SpecificStorage.Value = 1.0e-4
    
    # -----------------------------------------------------------------------------
    # Phases
    # -----------------------------------------------------------------------------
    
    hflow.Phase.Names = "water"
    
    hflow.Phase.water.Density.Type = "Constant"
    hflow.Phase.water.Density.Value = 1.0
    
    hflow.Phase.water.Viscosity.Type = "Constant"
    hflow.Phase.water.Viscosity.Value = 1.0
    
    # -----------------------------------------------------------------------------
    # Contaminants
    # -----------------------------------------------------------------------------
    
    hflow.Contaminants.Names = ""
    
    # -----------------------------------------------------------------------------
    # Gravity
    # -----------------------------------------------------------------------------
    
    hflow.Gravity = 1.0
    
    # -----------------------------------------------------------------------------
    # Setup timing info
    # -----------------------------------------------------------------------------
    
    hflow.TimingInfo.BaseUnit = 1.0
    hflow.TimingInfo.StartCount = -1
    hflow.TimingInfo.StartTime = 0.0
    hflow.TimingInfo.StopTime = 500.0
    hflow.TimingInfo.DumpInterval = -1
    
    # -----------------------------------------------------------------------------
    # Porosity
    # -----------------------------------------------------------------------------
    
    hflow.Geom.Porosity.GeomNames = "domain"
    hflow.Geom.domain.Porosity.Type = "Constant"
    hflow.Geom.domain.Porosity.Value = 0.390
    
    # -----------------------------------------------------------------------------
    # Domain
    # -----------------------------------------------------------------------------
    
    hflow.Domain.GeomName = "domain"
    
    # -----------------------------------------------------------------------------
    # Mobility
    # -----------------------------------------------------------------------------
    
    hflow.Phase.water.Mobility.Type = "Constant"
    hflow.Phase.water.Mobility.Value = 1.0
    
    # -----------------------------------------------------------------------------
    # Wells
    # -----------------------------------------------------------------------------
    
    hflow.Wells.Names = ""
    
    # -----------------------------------------------------------------------------
    # Time Cycles
    # -----------------------------------------------------------------------------
    
    hflow.Cycle.Names = "constant"
    hflow.Cycle.constant.Names = "alltime"
    hflow.Cycle.constant.alltime.Length = 1
    hflow.Cycle.constant.Repeat = -1
    
    # -----------------------------------------------------------------------------
    # Boundary Conditions: Pressure
    # -----------------------------------------------------------------------------
    
    hflow.BCPressure.PatchNames = "left right front back bottom top"
    
    # hflow.Patch.top.BCPressure.Type = "DirEquilRefPatch"
    # hflow.Patch.top.BCPressure.Cycle = "constant"
    # hflow.Patch.top.BCPressure.RefGeom = "domain"
    # hflow.Patch.top.BCPressure.RefPatch = "bottom"
    # hflow.Patch.top.BCPressure.alltime.Value = 1.0
    
    # hflow.Patch.top.BCPressure.Type          = 'OverlandFlow'
    # hflow.Patch.top.BCPressure.Cycle         = 'constant'
    # hflow.Patch.top.BCPressure.alltime.Value = -0.1

    hflow.Patch.top.BCPressure.Type          = 'FluxConst'
    hflow.Patch.top.BCPressure.Cycle         = 'constant'
    hflow.Patch.top.BCPressure.alltime.Value = -0.01
    
    # hflow.Patch.bottom.BCPressure.Type = "DirEquilRefPatch"
    # hflow.Patch.bottom.BCPressure.Cycle = "constant"
    # hflow.Patch.bottom.BCPressure.RefGeom = "domain"
    # hflow.Patch.bottom.BCPressure.RefPatch = "bottom"
    # hflow.Patch.bottom.BCPressure.alltime.Value = 0

    hflow.Patch.bottom.BCPressure.Type = "FluxConst"
    hflow.Patch.bottom.BCPressure.Cycle = "constant"
    hflow.Patch.bottom.BCPressure.alltime.Value = 0.0
    
    hflow.Patch.front.BCPressure.Type = "FluxConst"
    hflow.Patch.front.BCPressure.Cycle = "constant"
    hflow.Patch.front.BCPressure.alltime.Value = 0.0
    
    hflow.Patch.back.BCPressure.Type = "FluxConst"
    hflow.Patch.back.BCPressure.Cycle = "constant"
    hflow.Patch.back.BCPressure.alltime.Value = 0.0
    
    hflow.Patch.right.BCPressure.Type = "FluxConst"
    hflow.Patch.right.BCPressure.Cycle = "constant"
    hflow.Patch.right.BCPressure.alltime.Value = 0.0
    
    hflow.Patch.left.BCPressure.Type = "FluxConst"
    hflow.Patch.left.BCPressure.Cycle = "constant"
    hflow.Patch.left.BCPressure.alltime.Value = 0.0
    
    # ---------------------------------------------------------
    # Topo slopes in x-direction
    # ---------------------------------------------------------
    # topo slopes do not figure into the impes (fully sat) case but we still
    # need keys for them
    
    hflow.TopoSlopesX.Type = "Constant"
    hflow.TopoSlopesX.GeomNames = ""
    hflow.TopoSlopesX.Geom.domain.Value = 0.0
    
    # ---------------------------------------------------------
    # Topo slopes in y-direction
    # ---------------------------------------------------------
    
    hflow.TopoSlopesY.Type = "Constant"
    hflow.TopoSlopesY.GeomNames = ""
    
    hflow.TopoSlopesY.Geom.domain.Value = 0.0
    
    # ---------------------------------------------------------
    # Mannings coefficient
    # ---------------------------------------------------------
    # mannings roughnesses do not figure into the impes (fully sat) case but we still
    # need a key for them
    
    hflow.Mannings.Type = "Constant"
    hflow.Mannings.GeomNames = ""
    hflow.Mannings.Geom.domain.Value = 0.0
    
    # -----------------------------------------------------------------------------
    # Phase sources:
    # -----------------------------------------------------------------------------
    
    hflow.PhaseSources.water.Type = "Constant"
    hflow.PhaseSources.water.GeomNames = "domain"
    hflow.PhaseSources.water.Geom.domain.Value = 0.0
    
    #-----------------------------------------------------------------------------
    # Exact solution specification for error calculations
    #-----------------------------------------------------------------------------
    hflow.KnownSolution = 'NoKnownSolution'
    
    #-----------------------------------------------------------------------------
    # Relative Permeability
    #-----------------------------------------------------------------------------
    hflow.Phase.RelPerm.Type        = 'VanGenuchten'
    hflow.Phase.RelPerm.GeomNames   = 'domain'
    
    hflow.Geom.domain.RelPerm.Alpha = 1.0
    hflow.Geom.domain.RelPerm.N     = 2.0
    
    #---------------------------------------------------------
    # Saturation
    #---------------------------------------------------------
    hflow.Phase.Saturation.Type        = 'VanGenuchten'
    hflow.Phase.Saturation.GeomNames   = 'domain'
    
    hflow.Geom.domain.Saturation.Alpha = 1.0
    hflow.Geom.domain.Saturation.N     = 2.0
    hflow.Geom.domain.Saturation.SRes  = 0.2
    hflow.Geom.domain.Saturation.SSat  = 1.0
    
    #-----------------------------------------------------------------------------
    # Time Cycles
    #-----------------------------------------------------------------------------
    hflow.Cycle.Names = 'constant'
    hflow.Cycle.constant.Names = 'alltime'
    hflow.Cycle.constant.alltime.Length = 1
    hflow.Cycle.constant.Repeat = -1
    
    # Writing output options for ParFlow
    write_pfb = True  #only PFB output for water balance example
    #  PFB  no SILO
    hflow.Solver.PrintSubsurf             = True
    hflow.Solver.PrintSubsurfData         = write_pfb
    hflow.Solver.PrintPressure            = write_pfb
    hflow.Solver.PrintSaturation          = write_pfb
    hflow.Solver.PrintCLM                 = write_pfb
    hflow.Solver.PrintMask                = write_pfb
    hflow.Solver.PrintSpecificStorage     = write_pfb
    hflow.Solver.PrintEvapTrans           = write_pfb
    hflow.Solver.PrintVelocities          = True 
    
    # # -----------------------------------------------------------------------------
    # #  Solver Impes
    # # -----------------------------------------------------------------------------
    
    # hflow.Solver.MaxIter = 50
    # hflow.Solver.AbsTol = 1e-10
    # hflow.Solver.Drop = 1e-15
    
    #-----------------------------------------------------------------------------
    # Setup Timing
    #-----------------------------------------------------------------------------
    hflow.TimingInfo.BaseUnit     = 1.0
    hflow.TimingInfo.StartCount   = 0
    hflow.TimingInfo.StartTime    = 0.0
    hflow.TimingInfo.StopTime     = 200.0
    hflow.TimingInfo.DumpInterval = 1.0
    hflow.TimeStep.Type           = 'Constant'
    hflow.TimeStep.Value          = 1.0
    
    
    #-----------------------------------------------------------------------------
    # Set solver parameters
    #-----------------------------------------------------------------------------
    hflow.Solver         = 'Richards'
    hflow.Solver.MaxIter = 9000
    
    hflow.Solver.Nonlinear.MaxIter           = 200
    hflow.Solver.Nonlinear.ResidualTol       = 1e-5
    hflow.Solver.Nonlinear.EtaChoice         = 'Walker1'
    hflow.Solver.Nonlinear.EtaValue          = 0.01
    hflow.Solver.Nonlinear.UseJacobian       = True
    hflow.Solver.Nonlinear.DerivativeEpsilon = 1e-12
    hflow.Solver.Nonlinear.StepTol           = 1e-30
    hflow.Solver.Nonlinear.Globalization     = 'LineSearch'
    hflow.Solver.Linear.KrylovDimension      = 100
    hflow.Solver.Linear.MaxRestarts          = 5
    hflow.Solver.Linear.Preconditioner       = 'PFMG'
    hflow.Solver.PrintSubsurf                = True
    hflow.Solver.Drop                        = 1E-20
    hflow.Solver.AbsTol                      = 1E-9
    
    #---------------------------------------------------
    # Initial conditions: water pressure
    #---------------------------------------------------
    hflow.ICPressure.Type                 = 'Constant'
    hflow.ICPressure.GeomNames            = 'domain'
    hflow.Geom.domain.ICPressure.Value    = -10.00
    hflow.Geom.domain.ICPressure.RefGeom  = 'domain'
    hflow.Geom.domain.ICPressure.RefPatch = 'top'
    #This next cell sets the directory where ParFlow is installed and initiates a ParFlow run based on the keys defined above. Don't worry if parts of this (or all of it) seems intimidating or confusing; it takes time and practice to get used to anything new!
    
    
    # -----------------------------------------------------------------------------
    # Run and Unload the ParFlow output files
    # -----------------------------------------------------------------------------
    hflow.Geom.upper_aquifer.Perm.Seed = 33335
    
    hflow.run(working_directory=dir_name)

### Alpha = 1.0 (With capillarity)

In [ ]:
alpha_vG =1.0
time_array = [0, 1, 5, 100, 200]
correlated_random_field(alpha_vG)
plot_domain_corr_rnd(dir_name, "porosity", 0)
plot_domain_corr_rnd(dir_name, "z-permeability", 0)


# ==============================================================================
# ParFlow directory
#  - /home/SHARED/software/parflow/7c8e7f0
# ParFlow version
#  - 3.13.0
# Working directory
#  - /home/ms6985/ParFlow-project/Correlated-random-fields/test_output/harvey_flow
# ParFlow database
#  - harvey_flow.1.pfidb
# ==============================================================================




In [ ]:
import parflow.tools.hydrology as hydro
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from parflow.tools.fs import get_absolute_path, cp, rm, mkdir, exists
import parflow as pf

def surface_subsurface_storage(alpha_vG):
    run = Run.from_definition(f'{dir_name}/{run_name}.pfidb')
    data = run.data_accessor
    nt = len(data.times)
    nx = data.shape[2]
    ny = data.shape[1]
    nz = data.shape[0]
    dx = data.dx
    dy = data.dy
    dz = data.dz
    
    #print(nt,nx,ny,nz,dx,dy,dz)
    
    porosity = data.computed_porosity 
    specific_storage_1 = data.specific_storage
    specific_storage = np.ones_like(specific_storage_1)
    
    mannings = run.Mannings.Geom.domain.Value
    
    ## remove input filenames for TopoSlopes to force the data accessor to read the output slopes
    ## this fixes a windows issue
    run.TopoSlopesX.FileName = None
    run.TopoSlopesY.FileName = None
    
    #slopex = data.slope_x 
    #slopey = data.slope_y 
    mask = data.mask
    
    # formatting the mask so that values outside the domain are NA and inside the domain are 1
    nanmask=mask.copy()
    nanmask[nanmask == 0] = 'NaN'
    nanmask[nanmask > 0] = 1

    plt.contourf(data.mask[:,0,:])
    plt.colorbar()

    #Combining all outputs
    files = glob(f"{dir_name}/{run_name}.out.satur.*.pfb")
    saturation_arrays = pf.read_pfb_sequence(files)
    files = glob(f"{dir_name}/{run_name}.out.press.*.pfb")
    pressure_arrays = pf.read_pfb_sequence(files)

    subsurface_storage = np.zeros(nt+1)
    surface_storage = np.zeros(nt+1)
    data.time = 0

    for i, (pressure, saturation) in enumerate(zip(pressure_arrays, saturation_arrays)):
        subsurface_storage[i, ...] = np.sum(hydro.calculate_subsurface_storage(porosity, pressure, saturation, specific_storage, dx, dy, dz, mask = None),axis=(0, 1, 2))
        # total surface storage for this time step is the summation of substorage surface across all x/y slices
        surface_storage[i, ...] = np.sum( hydro.calculate_surface_storage(pressure, dx, dy, mask = None),axis=(0, 1))
        data.time = i
        #print(hydro.calculate_subsurface_storage(porosity, pressure, saturation, specific_storage, dx, dy, dz, mask = None))
        # print(np.sum(hydro.calculate_subsurface_storage(porosity, pressure, saturation, specific_storage, dx, dy, dz, mask = nanmask),axis=(0, 1, 2)))
        # print(np.sum( hydro.calculate_surface_storage(pressure, dx, dy, mask = nanmask),axis=(0, 1)))
        # print(i,np.sum(porosity[porosity<0.9]*saturation[porosity<0.9]*dx*dy*dz[0]*specific_storage[porosity<0.9]))
        # print(i,np.sum(pressure[porosity>0.95]*dx*dy))
        #print(porosity * saturation * dz[0] * dx * dy)
    #print(dx, dy, dz)
    # print(specific_storage_1)
    
    # print(subsurface_storage)
    # print(surface_storage)

    np.savez(f'Rnd_corr_fields_surface_subsurface_storage_alphavG{alpha_vG}',subsurface_storage,surface_storage)  
    
    #plotting components of water balance
    fig, axs = plt.subplots(2, sharex=True)
    fig.suptitle(f'Randomly correlated field, Alpha={alpha_vG}')
    axs[0].plot(subsurface_storage, linewidth=3, color = "green")
    axs[0].set_title("Subsurface Storage")
    axs[1].plot(surface_storage, linewidth=3, color = "blue")
    axs[1].set_title("Surface Storage")
    plt.xlabel('Simulated Hours')
    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.85, 
                        wspace=0.7, 
                        hspace=1.0)
    axs[1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    plt.savefig(f'Rnd_corr_fields_surface_subsurface_storage_alphavG{alpha_vG}.pdf',bbox_inches='tight', dpi = 600)

surface_subsurface_storage(alpha_vG)

In [ ]:
alpha_vG =1.0
for time in time_array:
    #Saturation and pressure at time t = 0 hours
    plot_domain_corr_rnd(dir_name, "press", time,alpha_vG); 
    plot_domain_corr_rnd(dir_name, "satur", time,alpha_vG)

In [ ]:
alpha_vG =100.0
correlated_random_field(alpha_vG)
for time in time_array:
    #Saturation and pressure at time t = 0 hours
    plot_domain_corr_rnd(dir_name, "press", time,alpha_vG); 
    plot_domain_corr_rnd(dir_name, "satur", time,alpha_vG)

In [ ]:
import numpy as np
#Difference the two contours
def difference_plot(time):
    #print(data)
    data = np.load(f'Rnd_corr_fields_satur_alphavG1.0_time{time}.npz')
    x_faces = data['arr_0']; z_faces = data['arr_1']; press1_data_array = data['arr_2']
    x_center = np.convolve(x_faces, [0.5, 0.5], "valid"); z_center = np.convolve(z_faces, [0.5, 0.5], "valid")

    x_array, z_array = np.meshgrid(x_center,z_center)
    print(np.shape(x_array),np.shape(z_array),np.shape(press1_data_array))
   
    # plt.figure()
    # plt.contourf(time_array_plot,z_array,press2_data_array,400, cmap='Blues',vmin=0,vmax=1.0)
    # plt.colorbar()
    # plt.show
    data = np.load(f'Rnd_corr_fields_satur_alphavG1.0_time{time}.npz');   satur1_data_array   = data['arr_2']
    data = np.load(f'Rnd_corr_fields_press_alphavG100.0_time{time}.npz'); press100_data_array = data['arr_2']
    data = np.load(f'Rnd_corr_fields_satur_alphavG100.0_time{time}.npz'); satur100_data_array = data['arr_2']
    
    variable_array = ["satur","press"] #satur or press
    
    for variable in variable_array:
        fig = plt.figure(figsize=(8,8) , dpi=100)
        if variable == "satur":
            label = "Saturation [-]"
            title = "Saturation [-]"
        elif variable == "press":
            label = "Pressure Head [m]"
            title = "Pressure Head [m]"
        elif variable == "porosity":
            label = "Porosity"
            title = "Porosity"
    
        if variable == "satur":
            im = plt.contourf(x_array,z_array,satur1_data_array-satur100_data_array,400, cmap='Blues',vmin=0,vmax=1.0,antialiased=False,edgecolor='face')
        else:
            im = plt.contourf(x_array,z_array,press1_data_array-press100_data_array,400, cmap='Blues',antialiased=False,edgecolor='face')        
        plt.xlabel(f"x [m]")
        cbar = fig.colorbar(im)
    
        cbar.ax.set_ylabel(f"Difference in {title}")
        plt.ylabel('z [m]')
        plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
        plt.legend(loc='best', frameon=False)
        plt.savefig(f'Rnd_corr_fields_{variable}_contour_difference_time{time}.pdf',bbox_inches='tight', dpi = 600)

In [ ]:
for time in time_array:
    #Saturation and pressure difference at time t = 0 hours
    difference_plot(time)